# Air Quality in Houston - Step 1: Data Wrangling#

## 1. Aim of the project and origin of the data ##

### 1.1. The Problem: ###
What is the future of indoor and outdoor air quality in Houston and its impact on Houstonians’ health?

Air quality has been a concern for Houston’s officials and population for several years. Houston’ s legendary around-the-clock traffic jam, its growing population, its humid subtropical climate conditions,   and the sprawling potpourri of pollutants released by refineries and chemical plants have made Space City’s air hard to breathe for a lot of Houstonians. In 2018, The Mayor's Task Force on the Health Effects of Air Pollution has identified 12 pollutants as definite health risks for Houstonians, the main one being Ozone.

In 2020, the plastic industry is growing rapidly, freeways are being widened to allow for more traffic to flow through, and more people are moving in. Houston  is currently the 5th largest metro population in the US with 6,997,384 inhabitants and is predicted to host 8.7 millions inhabitants by 2028 according to the Texas Demographic Center.  

Where is the air quality headed?

The aim of this capstone project is to predict the indoor and outdoor air quality in Houston for each upcoming decades up to 2050 using daily air data summaries, known potential drivers of air quality, and the city development forecast (i.e. population growth, change in land use...) from the Houston-Galveston Area Council. The impact of air quality on health will be presented by an overlay of AQI (Air Quality Index) calculated data and ELS (Effects Screening Levels). The analysis focuses on 6 pollutants of concern for Houston, namely Ozone (O3), Sulfur Dioxide (SO2), Carbon Monoxide (CO), Nitrogen Dioxide (NO2), and particulate matter (PM2.5 and PM10). In this project Air Quality refers to pollutant concentrations in the air.


### 1.2. The Data: ###
All the datasets used in this capstone were available online between 09/01/2020 and 09/15/2020 from the following websites:

- Relationships of Indoor, Outdoor, and Personal Air (RIOPA) dataset:
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/7UBE7P&version=1.0

- Daily Pollutant concentration measurements for Houston and effect screening levels from the Texas Commision On Environmental Quality (TECQ): https://www.tceq.texas.gov/ 

- Daily Pollutant concentration measurements for Houston from the U.S. Environmental Protection Agency: https://www.epa.gov/

- Weather daily summaries: https://www.noaa.gov/

- Land use data, population data and forecast from the Houston-Galveston Area Council: http://www.h-gac.com/home/default.aspx

- Road surface data: https://www.txdot.gov/inside-txdot/division/transportation-planning/roadway-inventory.html

- Traffic count, road size data, census data: https://cohgis-mycity.opendata.arcgis.com


### 1.3. Data and File Location On Github: ###
This project is hosted on Github in "Aurenkeelin18/TheFoxerine/CP2_AQ_HOUSTON". The folder is organized using subfolders:
- '00*_*OriginalData' contains all the original data files subdivided by role. 'AQ' contains air quality data and ancillary reports from TECQ and EPA. 'METEO' contains weather data from NOAA. 'RIOPA' contains the data from the indoor/outdoor stud from the RIOPA team.
- '00*_*SavedDataframes' contains dataframes saved in excel files.
- '00*_*StuffAndThings' contains images, maps or other miscellaneous items that were used in the project.
- '00*_*ZeCollection' contains clean copies of piece of coding used in this project that felt worth setting aside for future usage (i.e. functions, code for mapping something, awesome looking plots...)
- '01*_*DataWrangling' contains saved dataframes (pre-and post cleaning) and jupyter notebooks associated to the data wrangling.

Additional subfolders will be added as the project progresses through the DSM following the same nomenclature - i.e. '02*_*EDA', '03*_*Modelling'...- as well as the final report and presentation.

### 1.4. Getting Started ###
Below are found the libraries used for data wrangling as well as the path to all subfolders.

In [299]:
import numpy as np
import pandas as pd
import glob
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict,OrderedDict, namedtuple
import json

path_header='C:\\Users\\Anne\\Documents\\GIT\\TheFoxerine\\'
path_df='CP2_AQ_HOUSTON\\00_SavedDataframes\\'
path_riopa='CP2_AQ_HOUSTON\\00_OriginalData\\RIOPA\\'
path_meteo='CP2_AQ_HOUSTON\\00_OriginalData\\METEO\\'
path_AQ='CP2_AQ_HOUSTON\\00_OriginalData\\AQ\\'
path_demog='CP2_AQ_HOUSTON\\00_OriginalData\\DEMOG\\'

##2. Data Collection##
###2.1. Indoor/Outdoor Air Quality:###

In this section the relevant csv files from the RIOPA dataset are loaded and merged.

In [300]:
### Loading all files pertinent to the Indoor/Outdoor AQ analysis
riopa_pm25=pd.read_csv(path_header+path_riopa+'PM_Mass.csv')
riopa_temp=pd.read_csv(path_header+path_riopa+'TempRH.csv')
riopa_landuse=pd.read_csv(path_header+path_riopa+'Land_Use.csv')
riopa_aer=pd.read_csv(path_header+path_riopa+'AER.csv')
riopa_meteo=pd.read_csv(path_header+path_riopa+'met_avg_linkid.csv')
riopa_blockgroup=pd.read_csv(path_header+path_riopa+'CensusBlockGroup_Home.csv')
coh_blockgroup=pd.read_csv(path_header+path_demog+'COH_DEMOGRAPHICS.csv')

### let's have a look to the column names
riopa_list=[riopa_pm25,riopa_temp,riopa_landuse,riopa_aer,riopa_meteo,riopa_meteo_noaa,riopa_blockgroup,coh_blockgroup]
for name in riopa_list:
    print(name.info())   



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138 entries, 0 to 1137
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LinkID         1138 non-null   object 
 1   SampleID       1138 non-null   int64  
 2   HomeID         1138 non-null   object 
 3   Type           1138 non-null   object 
 4   PM25mass       1112 non-null   float64
 5   Validity       171 non-null    float64
 6   Mass_mg        1135 non-null   float64
 7   Volume_m3      1117 non-null   float64
 8   comments       1106 non-null   object 
 9   datestarted    1138 non-null   object 
 10  dateended      1138 non-null   object 
 11  PM25mass_flag  8 non-null      object 
dtypes: float64(4), int64(1), object(7)
memory usage: 106.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LinkID      11

LinkID is the primary key in the RIOPA relational database. It is a unique subject home identifier, first two digits state code (NJ,TX, CA), three digit numbers assigned to home in "Home ID"
during first visit, one digit number specifying the visit number (1 =first, 2=second), one digit sample type (lndoor=1,Outdoor=2, Personal Adult=3, Personal Child 1-4 ~ 4-7, Blank
=8, Control = 9, Vehicle =0), one digit for duplicate/QA code (Sample = 0, Duplicate =1, Repeat Analysis = 2, Backup of PUF sample for Breakthrough =3, Backup Duplicate=4). Only LinkID starting with TX will be kept.

The RIOPA datasets will be joined using the link ID and startdate. The weather daily summaries from NOAA (riopa*_*meteo*_*noaa) wil be joined to startdate with DATE.
As not all the data present in the datasets are relevant to the analysis, each dataset will be filtered prior to the merging.

In [301]:
### Filter riopa_pm25
f_riopa_pm25=riopa_pm25[['LinkID','SampleID','HomeID','Type','PM25mass','Validity','comments','datestarted','dateended']]

### Filter riopa_temp
f_riopa_temp=riopa_temp[['LinkID','ID','HomeID','Visit','Start_Date','Location','temp_c','avg_rh']]

### Filter riopa_landuse
f_riopa_landuse=riopa_landuse[['LinkID','Class']]

### Filter riopa_aer
f_riopa_aer=riopa_aer[['LinkID','HomeID','AER','comment','Start_Date','End_Date']]

### Filter riopa_meteo
f_riopa_meteo=riopa_meteo[['LinkID','datestarted','dateended','avg_Dry_bulb_temp','avg_Dew_point_temp','avg_Wet_bulb_temp','avg_RH']]

### Filter riopa_blockgroup
f_riopa_blockgroup=riopa_blockgroup[['LinkID', 'Census_Block_Group_Code']]

### Filter coh_blockgroup
f_coh_blockgroup=coh_blockgroup[['census_group_block_code', 'home_lat', 'home_long', 'geoid']]

I am renaming the columns of the dataframes below to make the merge/join coding easier to read and to keep the metadata intact (i.e. rename 'comment' and dates columns).

In [302]:
### Clarify column names
col_f_riopa_pm25={'LinkID':'linkid','SampleID':'sampleid','HomeID':'homeid','Type':'airtype','PM25mass':'pm25','Validity':'validity','comments':'comments_pm25','datestarted':'date_start_pm25','dateended':'date_end_pm25'}
fr_riopa_pm25=f_riopa_pm25.rename(columns=col_f_riopa_pm25)

col_f_riopa_temp={'LinkID':'linkid','ID':'tempid','HomeID':'homeid','Visit':'visitnumber','Start_Date':'date_temp','Location':'location','temp_c':'ambient_temp_c','avg_rh':'ambient_rh'}
fr_riopa_temp=f_riopa_temp.rename(columns=col_f_riopa_temp)

col_f_riopa_landuse={'LinkID':'linkid','Class':'landuse_class'}
fr_riopa_landuse=f_riopa_landuse.rename(columns=col_f_riopa_landuse)

col_f_riopa_aer={'LinkID':'linkid','HomeID':'homeid','AER':'airexrate','comment':'comment_aer','Start_Date':'date_start_aer','End_Date':'date_end_aer'}
fr_riopa_aer=f_riopa_aer.rename(columns=col_f_riopa_aer)

col_f_riopa_meteo={'LinkID':'linkid','datestarted':'date','dateended':'date_end_meteo','avg_Dry_bulb_temp':'temp_dry','avg_Dew_point_temp':'dew_point','avg_Wet_bulb_temp':'temp_wet','avg_RH':'rh'}
fr_riopa_meteo=f_riopa_meteo.rename(columns=col_f_riopa_meteo)

col_f_riopa_blockgroup={'LinkID':'linkid','Census_Block_Group_Code':'census_group_block_code'}
fr_riopa_blockgroup=f_riopa_blockgroup.rename(columns=col_f_riopa_blockgroup)

### coh_blockgroup is fine as is
fr_coh_blockgroup=f_coh_blockgroup

The RIOPA datasets contain data from Houston (TX), Elizabeth (NJ), and Los Angeles (CA). Only the texan data is of interest.

In [303]:
### Removing non texan data
tx_riopa_pm25=fr_riopa_pm25[fr_riopa_pm25.linkid.str.contains('TX')==True]
tx_riopa_temp=fr_riopa_temp[fr_riopa_temp.linkid.str.contains('TX')==True]
tx_riopa_landuse=fr_riopa_landuse[fr_riopa_landuse.linkid.str.contains('TX')==True]
tx_riopa_aer=fr_riopa_aer[fr_riopa_aer.linkid.str.contains('TX')==True]
tx_riopa_meteo=fr_riopa_meteo[fr_riopa_meteo.linkid.str.contains('TX')==True]

The last step before merging is to convert the dates from 'object' to datetime and to add a formal "date" column to dataframes where applicable.

In [304]:
### Converting date columns to datetime objects and adding a date column.
tx_riopa_pm25['date_start_pm25'] = pd.to_datetime(tx_riopa_pm25['date_start_pm25'])
tx_riopa_pm25['date_end_pm25'] = pd.to_datetime(tx_riopa_pm25['date_end_pm25'])
tx_riopa_pm25['date']=tx_riopa_pm25['date_start_pm25']

tx_riopa_temp['date_temp'] = pd.to_datetime(tx_riopa_temp['date_temp'])
tx_riopa_temp['date']=tx_riopa_temp['date_temp']

tx_riopa_aer['date_start_aer'] = pd.to_datetime(tx_riopa_aer['date_start_aer'])
tx_riopa_aer['date_end_aer'] = pd.to_datetime(tx_riopa_aer['date_end_aer'])
tx_riopa_aer['date']=tx_riopa_aer['date_start_aer']

tx_riopa_meteo['date'] = pd.to_datetime(tx_riopa_meteo['date'])
tx_riopa_meteo['date_end_meteo'] = pd.to_datetime(tx_riopa_meteo['date_end_meteo'])

C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

It turns out that the tx_riopa_meteo dataframe contains three measurements per sampling period that will create a duplication issue during merging (i.e. there will be 3 identical PM 2.5 measurement per given date).

In [305]:
tx_riopa_meteo.head()

,linkid,date,date_end_meteo,temp_dry,dew_point,temp_wet,rh
6817,TX001110,1999-06-23 22:53:00,1999-06-25 22:53:00,80.5143,76.0082,77.3286,86.7959
6818,TX001110,1999-06-23 22:53:00,1999-06-25 22:53:00,80.7653,76.2286,77.5306,86.9796
6819,TX001110,1999-06-23 22:53:00,1999-06-25 22:53:00,80.9184,76.1878,77.5592,86.3061
6820,TX001120,1999-06-23 22:53:00,1999-06-25 22:53:00,80.5143,76.0082,77.3286,86.7959
6821,TX001120,1999-06-23 22:53:00,1999-06-25 22:53:00,80.7653,76.2286,77.5306,86.9796


A non-indexed, groupby on the dataframe with an aggregation ('mean') will get unique records per date and linkid.

In [306]:
tx_riopa_meteo2=tx_riopa_meteo.groupby(['linkid','date'], as_index=False).agg({'temp_dry':'mean','dew_point':'mean','temp_wet': 'mean','rh':'mean'})
tx_riopa_meteo2['date']=tx_riopa_meteo2['date'].dt.date
tx_riopa_meteo2['date'] = pd.to_datetime(tx_riopa_meteo2['date'])
tx_riopa_meteo2.head()

,linkid,date,temp_dry,dew_point,temp_wet,rh
0,TX001110,1999-06-23,80.732667,76.141533,77.472800,86.693867
1,TX001120,1999-06-23,80.732667,76.141533,77.472800,86.693867
2,TX001130,1999-06-23,80.752400,76.197267,77.514967,86.809533
3,TX001140,1999-06-23,80.752400,76.197267,77.514967,86.809533
4,TX001210,2001-02-08,63.957467,57.645400,60.595767,80.602833


In [307]:
tx_riopa_meteo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 832 entries, 0 to 831
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   linkid     832 non-null    object        
 1   date       832 non-null    datetime64[ns]
 2   temp_dry   832 non-null    float64       
 3   dew_point  832 non-null    float64       
 4   temp_wet   832 non-null    float64       
 5   rh         832 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 45.5+ KB


The datasets are ready to be merged.

In [308]:
### merge landuse to pm25 to create riopa_m1
riopa_m1=pd.merge(tx_riopa_pm25,tx_riopa_landuse,how='left',on='linkid')

### merge riopa_m1 with tx_riopa_temp on linkid,homeid and date
riopa_m2=pd.merge(riopa_m1,tx_riopa_temp,how='left',on=['linkid','homeid','date'])

### merge riopa_m2 with tx_riopa_aer on linkid,homeid and date to create riopa_m3
riopa_m3=pd.merge(riopa_m2,tx_riopa_aer,how='left',on=['linkid','homeid','date'])

# merge riopa_m3 with tx_riopa_meteo on linkid to create riopa_m4
riopa_m4=pd.merge(riopa_m3,tx_riopa_meteo2,how='left',on=['linkid','date'])

# merge riopa_m4 with fr_riopa_blockgroup on linkid to create riopa_m5
riopa_m5=pd.merge(riopa_m4,fr_riopa_blockgroup,how='left',on='linkid')

# merge riopa_m5 with fr_coh_blockgroup on census_group_block_code to create riopa
riopa=pd.merge(riopa_m5,fr_coh_blockgroup,how='left',on='census_group_block_code')

In [309]:
riopa.head()

,linkid,sampleid,homeid,airtype,pm25,validity,comments_pm25,date_start_pm25,date_end_pm25,date,...,date_start_aer,date_end_aer,temp_dry,dew_point,temp_wet,rh,census_group_block_code,home_lat,home_long,geoid
0,TX103110,10951,EX103,INDOOR,15.2,NaN,\r,2000-10-17,2000-10-19,2000-10-17,...,2000-10-17,2000-10-19,69.283667,60.049633,63.702133,75.858133,4.820125e+11,29.792019,-95.081076,4.820130e+14
1,TX103110,10951,EX103,INDOOR,15.2,NaN,\r,2000-10-17,2000-10-19,2000-10-17,...,2000-10-17,2000-10-19,69.283667,60.049633,63.702133,75.858133,4.820125e+11,29.778960,-95.122067,4.820130e+14
2,TX103110,10951,EX103,INDOOR,15.2,NaN,\r,2000-10-17,2000-10-19,2000-10-17,...,2000-10-17,2000-10-19,69.283667,60.049633,63.702133,75.858133,4.820125e+11,29.789025,-95.094954,4.820130e+14
3,TX103110,10951,EX103,INDOOR,15.2,NaN,\r,2000-10-17,2000-10-19,2000-10-17,...,2000-10-17,2000-10-19,69.283667,60.049633,63.702133,75.858133,4.820125e+11,29.795403,-95.071440,4.820130e+14
4,TX103110,10951,EX103,INDOOR,15.2,NaN,\r,2000-10-17,2000-10-19,2000-10-17,...,2000-10-17,2000-10-19,69.283667,60.049633,63.702133,75.858133,4.820125e+11,29.782531,-95.113221,4.820130e+14


The relevant data from RIOPA has been merged together in the dataframe 'riopa'. Further processing of the data will be done later in the notebook.

###2.2. Weather Daily Summaries:###
The weather summaries from NOAA cover the daily recordings collected from three stations in the Houston area from 01/01/1999 to 12/31/2001 and 01/01/2008 to 09/01/2020. The stations are "Hobby Airport", "IAH" and "Galveston". Columns headers are identical from file to file. Only the 'date ' columns need to be converted to a datetime object prior to concatenating the files.

In [310]:
### Loading the files holding weather summaries
meteo_gal_2008_2012=pd.read_csv(path_header+path_meteo+'METEO_Galveston_2008_2012.csv')
meteo_gal_2013_2020=pd.read_csv(path_header+path_meteo+'METEO_Galveston_2013_2020.csv')
meteo_hob_2008_2012=pd.read_csv(path_header+path_meteo+'METEO_Houston_Hobby_2008_2012.csv')
meteo_hob_2013_2020=pd.read_csv(path_header+path_meteo+'METEO_Houston_Hobby_2013_2020.csv')
meteo_iah_2008_2012=pd.read_csv(path_header+path_meteo+'METEO_Houston_IAH_2008_2012.csv')
meteo_iah_2013_2020=pd.read_csv(path_header+path_meteo+'METEO_Houston_IAH_2013_2020.csv')
meteo_hob_1999_2001=pd.read_csv(path_header+path_meteo+'METEO_Houston_Hobby_1999_2001.csv')
meteo_iah_1999_2001=pd.read_csv(path_header+path_meteo+'METEO_Houston_IAH_1999_2001.csv')
meteo_gal_1999_2001=pd.read_csv(path_header+path_meteo+'METEO_Galveston_1999_2001.csv')

### Converting the "DATE" column from object to datetime
meteo_gal_2008_2012['DATE'] = pd.to_datetime(meteo_gal_2008_2012['DATE'])
meteo_gal_2013_2020['DATE'] = pd.to_datetime(meteo_gal_2013_2020['DATE'])
meteo_hob_2008_2012['DATE'] = pd.to_datetime(meteo_hob_2008_2012['DATE'])
meteo_hob_2013_2020['DATE'] = pd.to_datetime(meteo_hob_2013_2020['DATE'])
meteo_iah_2008_2012['DATE'] = pd.to_datetime(meteo_iah_2008_2012['DATE'])
meteo_iah_2013_2020['DATE'] = pd.to_datetime(meteo_iah_2013_2020['DATE'])
meteo_hob_1999_2001['DATE'] = pd.to_datetime(meteo_hob_1999_2001['DATE'])
meteo_iah_1999_2001['DATE'] = pd.to_datetime(meteo_iah_1999_2001['DATE'])
meteo_gal_1999_2001['DATE'] = pd.to_datetime(meteo_gal_1999_2001['DATE'])

# Concatenating all dataframes in one called meteo_all
meteo_all=pd.concat([meteo_gal_2008_2012,meteo_gal_2013_2020,meteo_gal_1999_2001,
                     meteo_hob_2008_2012,meteo_hob_2013_2020, meteo_hob_1999_2001,
                     meteo_iah_2008_2012,meteo_iah_2013_2020, meteo_iah_1999_2001])
meteo_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17533 entries, 0 to 1095
Data columns (total 74 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          17533 non-null  object        
 1   NAME             17533 non-null  object        
 2   LATITUDE         17533 non-null  float64       
 3   LONGITUDE        17533 non-null  float64       
 4   ELEVATION        17533 non-null  float64       
 5   DATE             17533 non-null  datetime64[ns]
 6   AWND             17264 non-null  float64       
 7   AWND_ATTRIBUTES  15803 non-null  object        
 8   PGTM             7455 non-null   float64       
 9   PGTM_ATTRIBUTES  5998 non-null   object        
 10  PRCP             17523 non-null  float64       
 11  PRCP_ATTRIBUTES  16062 non-null  object        
 12  SNOW             12806 non-null  float64       
 13  SNOW_ATTRIBUTES  11345 non-null  object        
 14  SNWD             12771 non-null  float6

Fortunately NOAA provides a key to decode the cryptic column names. The relevant columns are renamed and the dataframe is subset to create the final weather daily data table 'meteo'.

In [311]:
### Rename columns I want to keep
col_meteo={'STATION':'station_code', 'NAME':'station_name', 'LONGITUDE':'station_lon', 'LATITUDE':'station_lat', 'DATE':'date',  
           'TAVG':'temp_avg','TMIN':'temp_min','TMAX':'temp_max','WT08':'smoke_haze','PRCP':'rain_prcp','WT16':'rain','WT03':'thunder',
           'WT21':'fog_ground','WT02':'fog_heavy','WT01':'fog','AWND':'wind_avgspeed','PGTM':'wind_peak_gust_time','FMTM':'wind_time_fastest_mile',
           'WSF2':'wind_fastest_2min','WSF5':'wind_fastest_5min','WDF2':'wind_fastest_2min_direction','WDF5':'wind_fastest_5min_direction',
           'WT11':'wind_high_dmg','WT07':'dust_sand','WT13':'rain_mist','WT14':'drizzle','WT15':'drizzle_freezing','WT17':'rain_freezing',
           'WT18':'snow_grains','SNOW':'snow','SNWD':'snow_depth','WT09':'snow_drifting','WT04':'snow_sleet','WT05':'rain_hail',
           'WT06':'snow_glaze_rime','WT10':'wind_tornado'}
meteo_all=meteo_all.rename(columns=col_meteo)

### subsetting 
subsetlist=list({v for k, v in col_meteo.items()})
subsetlist.sort()
meteo=meteo_all[subsetlist]

Further processing of the dataframe meteo is done in Section 3.

### 2.3. Outdoor Air Quality:###
This is the biggest chunk of the data collection process where 91 files from EPA and 36 files from TECQ will be concatenated or merged. To speedup the process, loading functions are being used.

In [312]:
### Function glob_load which use glob to gather all the filenames
### in a folder and return the data into one dataframe
def glob_load(path='C:\\Users\\',file_header='EPA',file_pollutant='CO'):
    ### list of filenames
    allfiles = glob.glob(path + file_header+'_'+file_pollutant+'_*.csv')
    ### mydata list collect the data read by pd.read_csv
    mydata = []
    for filename in allfiles:
        df = pd.read_csv(filename, index_col=None, header=0)
        mydata.append(df)
    ### convert mydata list into a dataframe
    mydataframe = pd.concat(mydata, axis=0, ignore_index=True)
    ### return the dataframe
    return mydataframe

In [313]:
### Using glob load to get all the data from the EPA csv files
epa_co=glob_load(path=path_header+path_AQ+'\\',file_header='EPA',file_pollutant='CO')
epa_no2=glob_load(path=path_header+path_AQ+'\\',file_header='EPA',file_pollutant='NO2')
epa_so2=glob_load(path=path_header+path_AQ+'\\',file_header='EPA',file_pollutant='SO2')
epa_ozone=glob_load(path=path_header+path_AQ+'\\',file_header='EPA',file_pollutant='OZONE')
epa_pb=glob_load(path=path_header+path_AQ+'\\',file_header='EPA',file_pollutant='Pb')
epa_pm25=glob_load(path=path_header+path_AQ+'\\',file_header='EPA',file_pollutant='PM2_5')
epa_pm10=glob_load(path=path_header+path_AQ+'\\',file_header='EPA',file_pollutant='PM10')

In [314]:
epa_co.info()
epa_no2.info()
epa_so2.info()
epa_ozone.info()
epa_pb.info()
epa_pm25.info()
epa_pm10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20242 entries, 0 to 20241
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Date                               20242 non-null  object 
 1   Source                             20242 non-null  object 
 2   Site ID                            20242 non-null  int64  
 3   POC                                20242 non-null  int64  
 4   Daily Max 8-hour CO Concentration  20242 non-null  float64
 5   UNITS                              20242 non-null  object 
 6   DAILY_AQI_VALUE                    20242 non-null  int64  
 7   Site Name                          20242 non-null  object 
 8   DAILY_OBS_COUNT                    20242 non-null  int64  
 9   PERCENT_COMPLETE                   20242 non-null  float64
 10  AQS_PARAMETER_CODE                 20242 non-null  int64  
 11  AQS_PARAMETER_DESC                 20242 non-null  obj

EPA columns look good. The columns 'DATE' need to be converted to a datetime object. Some columns will have to be renamed to facilitate the merging of all these dataframes into one.

In [315]:
### dictionaries to rename columns 
col_epa_ozone={'Date':'date', 'Source':'ozone_8hr_source','Site ID':'site_id','POC':'ozone_8hr_poc',
               'Daily Max 8-hour Ozone Concentration':'ozone_8hr_max', 'UNITS':'ozone_8hr_units',
               'Site Name':'site_name', 'DAILY_OBS_COUNT':'ozone_8hr_obs_count', 
               'SITE_LATITUDE':'site_lat','SITE_LONGITUDE':'site_lon'}

col_epa_co={'Date':'date', 'Source':'co_8hr_source','POC':'co_8hr_poc','Site ID':'site_id',
            'Daily Max 8-hour CO Concentration':'co_8hr_max', 'UNITS':'co_8hr_units',
            'Site Name':'site_name', 'DAILY_OBS_COUNT':'co_8hr_obs_count',
            'SITE_LATITUDE':'site_lat', 'SITE_LONGITUDE':'site_lon'}
     
    
col_epa_no2={'Date':'date', 'Source':'no2_1hr_source','Site ID':'site_id','POC':'no2_1hr_poc',
             'Daily Max 1-hour NO2 Concentration':'no2_1hr_max', 'UNITS':'no2_1hr_units',
             'Site Name':'site_name', 'DAILY_OBS_COUNT':'no2_1hr_obs_count',
             'SITE_LATITUDE':'site_lat','SITE_LONGITUDE':'site_lon'}

col_epa_so2={'Date':'date', 'Source':'so2_1hr_source','Site ID':'site_id','POC':'so2_1hr_poc',
             'Daily Max 1-hour SO2 Concentration':'so2_1hr_max', 'UNITS':'so2_1hr_units',  
             'Site Name':'site_name', 'DAILY_OBS_COUNT':'so2_1hr_obs_count', 
             'SITE_LATITUDE':'site_lat','SITE_LONGITUDE':'site_lon'}
    
col_epa_pb={'Date':'date', 'Source':'pb_24hr_source','Site ID':'site_id','POC':'pb_24hr_poc',
            'Daily Mean Pb Concentration':'pb_24hr_mean','UNITS':'pb_24hr_units',
            'Site Name':'site_name', 'DAILY_OBS_COUNT':'pb_24hr_obs_count', 
            'SITE_LATITUDE':'site_lat','SITE_LONGITUDE':'site_lon'}
    
col_epa_pm25={'Date':'date', 'Source':'pm25_24hr_source','Site ID':'site_id','POC':'pm25_24hr_poc',
              'Daily Mean PM2.5 Concentration':'pm25_24hr_mean','UNITS':'pm25_24hr_units',
              'Site Name':'site_name', 'DAILY_OBS_COUNT':'pm25_24hr_obs_count',
              'SITE_LATITUDE':'site_lat','SITE_LONGITUDE':'site_lon'}

col_epa_pm10={'Date':'date','Source':'pm10_24hr_source','Site ID':'site_id','POC':'pm10_24hr_poc',
              'Daily Mean PM10 Concentration':'pm10_24hr_mean', 'UNITS':'pm10_24hr_units', 
              'Site Name':'site_name', 'DAILY_OBS_COUNT':'pm10_24hr_obs_count',
              'SITE_LATITUDE':'site_lat','SITE_LONGITUDE':'site_lon'}

### rename columns
repa_ozone=epa_ozone.rename(columns=col_epa_ozone)
repa_co=epa_co.rename(columns=col_epa_co)
repa_no2=epa_no2.rename(columns=col_epa_no2)
repa_so2=epa_so2.rename(columns=col_epa_so2)
repa_pb=epa_pb.rename(columns=col_epa_pb)
repa_pm25=epa_pm25.rename(columns=col_epa_pm25)
repa_pm10=epa_pm10.rename(columns=col_epa_pm10)

# convert date to datetime
repa_ozone['date'] = pd.to_datetime(repa_ozone['date'])
repa_co['date'] = pd.to_datetime(repa_co['date'])
repa_no2['date'] = pd.to_datetime(repa_no2['date'])
repa_so2['date'] = pd.to_datetime(repa_so2['date'])
repa_pb['date'] = pd.to_datetime(repa_pb['date'])
repa_pm25['date'] = pd.to_datetime(repa_pm25['date'])
repa_pm10['date'] = pd.to_datetime(repa_pm10['date'])

### subsetting
newcol_ozone=col_epa_ozone.values()
subset_ozone=[name for name in newcol_ozone]
frepa_ozone=repa_ozone[subset_ozone]

newcol_co=col_epa_co.values()
subset_co=[name for name in newcol_co]
frepa_co=repa_co[subset_co]

newcol_no2=col_epa_no2.values()
subset_no2=[name for name in newcol_no2]
frepa_no2=repa_no2[subset_no2]

newcol_so2=col_epa_so2.values()
subset_so2=[name for name in newcol_so2]
frepa_so2=repa_so2[subset_so2]

newcol_pb=col_epa_pb.values()
subset_pb=[name for name in newcol_pb]
frepa_pb=repa_pb[subset_pb]

newcol_pm25=col_epa_pm25.values()
subset_pm25=[name for name in newcol_pm25]
frepa_pm25=repa_pm25[subset_pm25]

newcol_pm10=col_epa_pm10.values()
subset_pm10=[name for name in newcol_pm10]
frepa_pm10=repa_pm10[subset_pm10]

Let's merge the epa dataset.

In [316]:
### merge in order ozone, co, no2, so2, pb, pm10, pm25
epa1=pd.merge(frepa_ozone,frepa_co,how='outer',on=['date','site_id','site_name','site_lat','site_lon'])
epa2=pd.merge(epa1,frepa_no2,how='outer',on=['date','site_id','site_name','site_lat','site_lon'])
epa3=pd.merge(epa2,frepa_so2,how='outer',on=['date','site_id','site_name','site_lat','site_lon'])
epa4=pd.merge(epa3,frepa_pb,how='outer',on=['date','site_id','site_name','site_lat','site_lon'])
epa5=pd.merge(epa4,frepa_pm10,how='outer',on=['date','site_id','site_name','site_lat','site_lon'])
epa=pd.merge(epa5,frepa_pm25,how='outer',on=['date','site_id','site_name','site_lat','site_lon'])

The second chunk of air quality data comes from TECQ and is composed of txt files which all have a header summary. To speed up loading of multiples files, another glob function is being used, called 'glob*_*load*_*txt'.

In [317]:
### Function glob_load_txt which use glob to gather all the filenames
### in a folder and return the data into one dataframe
def glob_load_txt(path='C:\\Users\\',file_header='EPA',file_pollutant='CO'):
    ### list of filenames
    allfiles = glob.glob(path + file_header+'_'+file_pollutant+'_*.txt')
    ### mydata list collect the data read by pd.read_table
    mydata = []
    for filename in allfiles:
        ### read_table must skip the header summary, hence the skiprows=9
        df = pd.read_table(filename,header=0,skiprows=9,sep='\t')
        mydata.append(df)
    ### convert mydata list into a dataframe
    mydataframe = pd.concat(mydata, axis=0, ignore_index=True)
    ### return the dataframe
    return mydataframe

### Using glob_load_txt to load the data
tamis_co=glob_load_txt(path=path_header+path_AQ+'\\',file_header='TAMIS',file_pollutant='CO')
tamis_no2=glob_load_txt(path=path_header+path_AQ+'\\',file_header='TAMIS',file_pollutant='NO2')
tamis_so2=glob_load_txt(path=path_header+path_AQ+'\\',file_header='TAMIS',file_pollutant='SO2')
tamis_ozone=glob_load_txt(path=path_header+path_AQ+'\\',file_header='TAMIS',file_pollutant='Ozone')

### Using just read_table for single files
tamis_pm10_24hr=pd.read_table(path_header+path_AQ+'\\TAMIS_PM10_24HR.txt',header=0,skiprows=9,sep='\t')
tamis_pm10_detail=pd.read_table(path_header+path_AQ+'\\TAMIS_PM10_2008_2020.txt',header=0,skiprows=9,sep='\t')
tamis_pm25=pd.read_table(path_header+path_AQ+'\\TAMIS_PM2_5_24HR_2008_2020.txt',header=0,skiprows=9,sep='\t')

C:\Users\Anne\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Anne\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (107) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [318]:
### An example of output
tamis_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302264 entries, 0 to 1302263
Data columns (total 19 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   State                 1302264 non-null  object 
 1   Region                1302264 non-null  object 
 2   County                1302264 non-null  object 
 3   City                  1302264 non-null  object 
 4   AQS Code              1302264 non-null  int64  
 5   Site Name             1302264 non-null  object 
 6   Latitude              1302264 non-null  float64
 7   Longitude             1302264 non-null  float64
 8   Year                  1302264 non-null  int64  
 9   Month                 1302264 non-null  int64  
 10  Day                   1302264 non-null  int64  
 11  Date                  1302264 non-null  int64  
 12  Start Hour            1302264 non-null  int64  
 13  Start Minute          1302264 non-null  int64  
 14  Start Time            1302264 non-

The TECQ*/*Tamis dataframes require some adjustement before merging. 
- The 'Date' column has to be converted into datetime. 
- Lead (Pb) concentrations are included in tamis_pm10_detail and therefore has to be retrieved.
- TECQ provides hourly measurements of ozone and carbon monoxide concentrations. The maximum values for each 8 hour period has to be extracted. This task will be done later on because there some interest at keeping the hourly data for modelling.


In [319]:
### Converting date format from yyyymmdd to datetime yyyy-mm-dd

tamis_ozone['date'] = pd.to_datetime(tamis_ozone['Date'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tamis_ozone['date']=pd.to_datetime(tamis_ozone['date'])
tamis_co['date'] = pd.to_datetime(tamis_co['Date'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tamis_co['date']=pd.to_datetime(tamis_co['date'])
tamis_no2['date'] = pd.to_datetime(tamis_no2['Date'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tamis_no2['date']=pd.to_datetime(tamis_no2['date'])
tamis_so2['date'] = pd.to_datetime(tamis_so2['Date'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tamis_so2['date']=pd.to_datetime(tamis_so2['date'])
tamis_pm10_24hr['date'] = pd.to_datetime(tamis_pm10_24hr['Date'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tamis_pm10_24hr['date']=pd.to_datetime(tamis_pm10_24hr['date'])
tamis_pm25['date'] = pd.to_datetime(tamis_pm25['Date'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tamis_pm25['date']=pd.to_datetime(tamis_pm25['date'])
tamis_pm10_detail['date'] = pd.to_datetime(tamis_pm10_detail['Date'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tamis_pm10_detail['date']=pd.to_datetime(tamis_pm10_detail['date'])


In [320]:
###  renaming columns

col_tamis_ozone={'City':'city', 'AQS Code':'site_code', 'Site Name':'site_name',
                 'Latitude':'site_lat', 'Longitude':'site_lon','Start Hour':'start_hour',
                 'Start Time':'start_time','Duration':'duration','SOC':'ozone_soc',
                 'Ozone (ppbv) <44201>':'ozone_1hr', 'date':'date'}

col_tamis_co={'City':'city', 'AQS Code':'site_code', 'Site Name':'site_name',
                 'Latitude':'site_lat', 'Longitude':'site_lon','Start Hour':'start_hour',
                 'Start Time':'start_time','Duration':'duration','SOC':'co_soc',
                 'Carbon Monoxide (ppmv) <42101>':'co_1hr', 'date':'date'}

col_tamis_no2={'City':'city', 'AQS Code':'site_code', 'Site Name':'site_name',
                 'Latitude':'site_lat', 'Longitude':'site_lon','Start Hour':'start_hour',
                 'Start Time':'start_time','Duration':'duration','SOC':'no2_soc',
                 'Nitrogen Dioxide (NO2) (ppbv) <42602>':'no2_1hr', 'date':'date'}

col_tamis_so2={'City':'city', 'AQS Code':'site_code', 'Site Name':'site_name',
                 'Latitude':'site_lat', 'Longitude':'site_lon','Start Hour':'start_hour',
                 'Start Time':'start_time','Duration':'duration','SOC':'so2_soc',
                 'Sulfur Dioxide (ppbv) <42401>':'so2_1hr', 'date':'date'}

col_tamis_pm10_24hr={'City':'city', 'AQS Code':'site_code', 'Site Name':'site_name',
                 'Latitude':'site_lat', 'Longitude':'site_lon','Start Hour':'start_hour',
                 'Start Time':'start_time','Duration':'duration','SOC':'pm10_24hr_soc',
                     'Pm10 - Lc (ug/m3 (LC)) <85101>':'pm10_24hr','Pm10 Total 0-10um Stp (ug/m3 (25 C)) <81102>':'pm10_total_24hr',
                     'Pm10-2.5 - Local Conditions (ug/m3 (LC)) <86101>':'pm10_minus_pm2_5_24hr','date':'date'}

col_tamis_pm25={'City':'city', 'AQS Code':'site_code', 'Site Name':'site_name',
                 'Latitude':'site_lat', 'Longitude':'site_lon','Start Hour':'start_hour',
                 'Start Time':'start_time','Duration':'duration','SOC':'pm25_soc',
                 'Pm2.5 - Local Conditions (ug/m3 (LC)) <88101>':'pm25_24hr', 'date':'date'}
     
col_tamis_pm10_detail={'City':'city', 'AQS Code':'site_code', 'Site Name':'site_name',
                 'Latitude':'site_lat', 'Longitude':'site_lon','Start Hour':'start_hour',
                 'Start Time':'start_time','Duration':'duration',
                 'Lead Pm10 Stp (ug/m3 (25 C)) <82128>':'pb_24hr', 'date':'date'}

rtamis_ozone=tamis_ozone.rename(columns=col_tamis_ozone)
rtamis_co=tamis_co.rename(columns=col_tamis_co)
rtamis_no2=tamis_no2.rename(columns=col_tamis_no2)
rtamis_so2=tamis_so2.rename(columns=col_tamis_so2)
rtamis_pm10_24hr=tamis_pm10_24hr.rename(columns=col_tamis_pm10_24hr)
rtamis_pm25=tamis_pm25.rename(columns=col_tamis_pm25)
rtamis_pm10_detail=tamis_pm10_detail.rename(columns=col_tamis_pm10_detail)

In [321]:
### subsetting
newcol_tamis_ozone=col_tamis_ozone.values()
sub_ozone=[name for name in newcol_tamis_ozone]
frtamis_ozone=rtamis_ozone[sub_ozone]

newcol_tamis_co=col_tamis_co.values()
sub_co=[name for name in newcol_tamis_co]
frtamis_co=rtamis_co[sub_co]

newcol_tamis_no2=col_tamis_no2.values()
sub_no2=[name for name in newcol_tamis_no2]
frtamis_no2=rtamis_no2[sub_no2]

newcol_tamis_so2=col_tamis_so2.values()
sub_so2=[name for name in newcol_tamis_so2]
frtamis_so2=rtamis_so2[sub_so2]

newcol_tamis_pm10_24hr=col_tamis_pm10_24hr.values()
sub_pm10_24hr=[name for name in newcol_tamis_pm10_24hr]
frtamis_pm10_24hr=rtamis_pm10_24hr[sub_pm10_24hr]

newcol_tamis_pm25=col_tamis_pm25.values()
sub_pm25=[name for name in newcol_tamis_pm25]
frtamis_pm25=rtamis_pm25[sub_pm25]

### subsetting Lead data
newcol_tamis_pm10_detail=col_tamis_pm10_detail.values()
sub_pm10_detail=[name for name in newcol_tamis_pm10_detail]
frtamis_pb=rtamis_pm10_detail[sub_pm10_detail]
frtamis_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302264 entries, 0 to 1302263
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   city        1302264 non-null  object        
 1   site_code   1302264 non-null  int64         
 2   site_name   1302264 non-null  object        
 3   site_lat    1302264 non-null  float64       
 4   site_lon    1302264 non-null  float64       
 5   start_hour  1302264 non-null  int64         
 6   start_time  1302264 non-null  object        
 7   duration    1302264 non-null  object        
 8   ozone_soc   1302264 non-null  int64         
 9   ozone_1hr   1302264 non-null  float64       
 10  date        1302264 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(4)
memory usage: 109.3+ MB


The calculation of the Air Quality Index requires, among other parameters, a 8-hour measurement of the concentration of ozone. EPA ozone data contains the maximum 8 hour ozone measurement for each day. Let's take the maximum measurement of ozone from tamis for each day.

In [322]:
### Pick the max ozone for each day
ozone_max=frtamis_ozone.groupby(['date','site_code'], as_index=False).agg({'ozone_1hr':'max'})
ozone_max['date'] = pd.to_datetime(ozone_max['date'])
ozone_max=ozone_max.rename(columns={'ozone_1hr':'ozone_8hr_max'})

### Merge the ozone_8hr_max column to the initial dataframe
mfrtamis_ozone=pd.merge(frtamis_ozone,ozone_max,how='left',on=['date','site_code'])
mfrtamis_ozone.columns

### Drop useless columns and duplicates
mfrtamis_ozone.drop(columns=['start_hour', 'start_time', 'duration', 'ozone_soc', 'ozone_1hr'], inplace=True)
mfrtamis_ozone.drop_duplicates(inplace=True)
mfrtamis_ozone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55526 entries, 0 to 1302131
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   city           55526 non-null  object        
 1   site_code      55526 non-null  int64         
 2   site_name      55526 non-null  object        
 3   site_lat       55526 non-null  float64       
 4   site_lon       55526 non-null  float64       
 5   date           55526 non-null  datetime64[ns]
 6   ozone_8hr_max  55526 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 3.4+ MB


This operation trimmed down the ozone dataset from tamis from 1,302,131 entries to 55,526. Further processing of the tamis dataframes will occur in section 3.

### 2.4. Monitoring Stations Information:###
Information about each weather station are extracted to match their location to interesting attributes such as landuse, source emission...etc..

In [323]:
### Monitoring stations from EPA
epa_stations_ozone=frepa_ozone[['site_id','site_name','site_lat','site_lon']]
epa_stations_co=frepa_co[['site_id','site_name','site_lat','site_lon']]
epa_stations_no2=frepa_no2[['site_id','site_name','site_lat','site_lon']]
epa_stations_so2=frepa_so2[['site_id','site_name','site_lat','site_lon']]
epa_stations_pm10=frepa_pm10[['site_id','site_name','site_lat','site_lon']]
epa_stations_pm25=frepa_pm25[['site_id','site_name','site_lat','site_lon']]
epa_stations_pb=frepa_pb[['site_id','site_name','site_lat','site_lon']]

epa_stations_ozone['poll']='ozone'
epa_stations_co['poll']='co'
epa_stations_no2['poll']='no2'
epa_stations_so2['poll']='so2'
epa_stations_pm10['poll']='pm10'
epa_stations_pm25['poll']='pm25'
epa_stations_pb['poll']='pb'

epa_stations_ozone.drop_duplicates(inplace=True)
epa_stations_co.drop_duplicates(inplace=True)
epa_stations_no2.drop_duplicates(inplace=True)
epa_stations_so2.drop_duplicates(inplace=True)
epa_stations_pm10.drop_duplicates(inplace=True)
epa_stations_pm25.drop_duplicates(inplace=True)
epa_stations_pb.drop_duplicates(inplace=True)

epa_stations=pd.concat([epa_stations_ozone,epa_stations_co,epa_stations_no2,epa_stations_so2,epa_stations_pm10,epa_stations_pm25,epa_stations_pb], axis=0)
epa_stations.drop_duplicates(inplace=True)

### Monitoring Stations from TECQ/Tamis
tamis_station_ozone=mfrtamis_ozone[['city','site_code','site_lat','site_lon']]
tamis_station_co=frtamis_co[['city','site_code','site_lat','site_lon']]
tamis_station_no2=frtamis_no2[['city','site_code','site_lat','site_lon']]
tamis_station_so2=frtamis_so2[['city','site_code','site_lat','site_lon']]
tamis_station_pm10_24hr=frtamis_pm10_24hr[['city','site_code','site_lat','site_lon']]
tamis_station_pm25=frtamis_pm25[['city','site_code','site_lat','site_lon']]
tamis_station_pb=frtamis_pb[['city','site_code','site_lat','site_lon']]

tamis_station_ozone.drop_duplicates(inplace=True)
tamis_station_co.drop_duplicates(inplace=True)
tamis_station_no2.drop_duplicates(inplace=True)
tamis_station_so2.drop_duplicates(inplace=True)
tamis_station_pm10_24hr.drop_duplicates(inplace=True)
tamis_station_pm25.drop_duplicates(inplace=True)
tamis_station_pb.drop_duplicates(inplace=True)

tamis_station_ozone['poll']='ozone'
tamis_station_co['poll']='co'
tamis_station_no2['poll']='no2'
tamis_station_so2['poll']='so2'
tamis_station_pm10_24hr['poll']='pm10'
tamis_station_pm25['poll']='pm25'
tamis_station_pb['poll']='pb'

tamis_stations=pd.concat([tamis_station_ozone,tamis_station_co,tamis_station_no2,tamis_station_so2,tamis_station_pm10_24hr,tamis_station_pm25,tamis_station_pb],axis=0)
tamis_stations.drop_duplicates(inplace=True)


C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

The exported xlsx files containing stations information that is plotted with all the shape files provided by the HGA tool to extract population data, landuse, roads...etc... The completed tables will be reimported in the EDA jupyter notebook. The mapping work will be*/*is stored in the folder '00*_*StuffAndThings' when polished.

##3. Data Definition and Cleaning:## 
###3.1. Location of Air Quality Stations:
Logically the cleaning has to start by looking at duplicate stations between EPA and TECQ before looking at duplicate within the air quality data. 

In [324]:
epa_solo=epa_stations.drop('poll',axis=1)
tamis_solo=tamis_stations.drop('poll',axis=1)
print(epa_solo.shape,tamis_solo.shape)

(79, 4) (58, 4)


There are 79 EPA stations and 58 TECQ stations. As TECQ reports data annually to EPA, there might be an overlap. Let's use the option 'indicator' of pd.merge to find out.

In [325]:
### comparing epa_stations to tamis_stations
allstations= epa_stations.merge(tamis_stations, indicator=True,how='outer')
allstations._merge[allstations._merge=='both'].count()                        

22

As indicated by the presence of 'both', there are 22 redundent stations. Let's find out which ones and store this information in the dataframe 'redund_stations'. This dataframe will be used later during the cleaning of air quality data. Here is a preview of redund_stations.

In [326]:
redund_stations=allstations.loc[allstations['_merge']=='both']
redund_stations.head()

,site_id,site_name,site_lat,site_lon,poll,city,site_code,_merge
7,482010047.0,Lang,29.834167,-95.489167,ozone,Houston,482010047.0,both
8,482010051.0,Houston Croquet,29.623889,-95.474167,ozone,Houston,482010051.0,both
10,482010062.0,Houston Monroe,29.625556,-95.267222,ozone,Houston,482010062.0,both
11,482010066.0,Houston Westhollow,29.723333,-95.635833,ozone,Houston,482010066.0,both
13,482010075.0,Houston Texas Avenue,29.752778,-95.350278,ozone,Houston,482010075.0,both


Another fact to keep in mind, as shown below, is that some stations measure multiple types of pollutant, and some measure only one pollutant.

In [327]:
#byloc=pd.melt(allstations,id_vars=['poll'],value_vars=['site_lat','site_lon'])
#byloc.head()
byloc=allstations.groupby(['site_lat','site_lon']).agg({'poll':'count'})
byloc

,,poll
site_lat,site_lon,
29.043759,-95.472946,2
29.254474,-94.861289,3
29.384444,-94.930833,1
29.385234,-94.931520,1
29.520443,-95.392509,2
29.583047,-95.015544,4
29.583333,-95.105000,1
29.589444,-95.353611,1
29.623889,-95.474167,2


To summarize the available information regarding air quality stations:
- the dataframes epa_stations and tamis_stations contain the information about the stations.
- the dataframe 'allstations' contain both datasets and a column '_merge' which indicates duplicate stations between both sets.
- 22 stations are common to both EPA and TECQ datasets. There are saved under 'redund_stations'
- the column 'poll' provides the type of measurement done at each station.
- some stations measure multiple pollutants.

###3.2. Meteorology Data:
Houston is a very large city. The weather tends to vary from one side to the other. With this in mind, the weather data was collected from three far apart stations: Hobby airport to the North, IAH to the South, and the Galveston station to the East.This hypothesis will have to be chekced after cleaning the data.
The method 'describe' shows that some data is numerical (i.e. temp, wind speed), some data is boolean (format 1 or Nan) and some data is informational (i.e. direction of wind). The dataset needs to be simplified into fewer columns showing the occurrence or not of an average meteorological event (i.e. rain, snow, wind, smoke, fog), some important numerical values (i.e. temperature, wind speed, rain amount) and of course date and the location of the weather stations

In [328]:
meteo.describe()

,drizzle,drizzle_freezing,dust_sand,fog,fog_ground,fog_heavy,rain,rain_freezing,rain_hail,rain_mist,...,thunder,wind_avgspeed,wind_fastest_2min,wind_fastest_2min_direction,wind_fastest_5min,wind_fastest_5min_direction,wind_high_dmg,wind_peak_gust_time,wind_time_fastest_mile,wind_tornado
count,123.0,4.0,139.0,6536.0,185.0,908.0,2354.0,4.0,575.0,2359.0,...,2710.0,17264.000000,17296.000000,17296.000000,17198.000000,17197.000000,25.0,7455.000000,6385.000000,10.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,8.646697,18.961153,166.340772,24.307856,165.760307,1.0,1371.543662,1575.029287,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.802111,5.556988,95.502780,7.224481,95.631063,0.0,538.049793,1245.755801,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.000000,2.900000,10.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,5.820000,15.000000,110.000000,19.900000,110.000000,1.0,1123.000000,1214.000000,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,8.050000,17.900000,150.000000,23.000000,150.000000,1.0,1434.000000,1513.000000,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,10.960000,21.900000,200.000000,28.000000,200.000000,1.0,1705.000000,1730.000000,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,36.910000,91.900000,360.000000,221.900000,360.000000,1.0,2359.000000,9999.000000,1.0


In [329]:
### Cleaning meteo
dicofog={'fog':'new','fog_ground':'new','fog_heavy':'new'}
meteo['fog_all']=meteo.groupby(dicofog,axis=1).sum()
meteo['fog_all'].replace([1,2,3],1,inplace=True)

dicosnow={'snow_sleet':'new','snow_grains':'new','drizzle_freezing':'new','snow':'new',
        'snow_drifting':'new','snow_glaze_rime':'new','rain_freezing':'new'}
meteo['snow_all']=meteo.groupby(dicofog,axis=1).sum()
meteo['snow_all'].replace([1,2,3,4,5,6,7,8],1,inplace=True)

dicorain={'rain':'new','drizzle':'new','rain_mist':'new','drizzle':'new',
        'rain_hail':'new'}
meteo['rain_all']=meteo.groupby(dicofog,axis=1).sum()
meteo['rain_all'].replace([1,2,3,4,5,6,7,8],1,inplace=True)

dicowind_dmg={'wind_tornado':'new','wind_high_dmg':'new'}
meteo['wind_dmg']=meteo.groupby(dicofog,axis=1).sum()
meteo['wind_dmg'].replace([1,2,3,4,5,6,7,8],1,inplace=True)

meteo_simple=meteo[['date', 'dust_sand','rain_prcp', 'smoke_haze', 'station_code',
       'station_lat', 'station_lon', 'station_name', 'temp_avg', 'temp_max',
       'temp_min', 'thunder', 'wind_avgspeed', 'wind_fastest_2min',
       'fog_all', 'snow_all', 'rain_all', 'wind_dmg']]

### Replace Nan by 0 in no numerical columns
meteo_simple['dust_sand'] = meteo_simple['dust_sand'].fillna(0)
meteo_simple['smoke_haze'] = meteo_simple['smoke_haze'].fillna(0)
meteo_simple['thunder'] = meteo_simple['thunder'].fillna(0)


C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Anne\anaconda3\lib\site-packages\pandas\core\series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\Anne\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

The temperature columns have some Nan values. It looks like it would be better to drop the rows without temp_min and temp_max, and to replace the NaN values in the temp*_*avg column by calculating an average from temp*_*min and temp*_*max.

In [330]:
meteo_simple['temp_min'].isnull().sum()

40

In [331]:
meteo_simple['temp_max'].isnull().sum()

39

In [332]:
meteo_simple['temp_avg'].isnull().sum()

3975

In [333]:
### Drop NaN in temperature columns
meteo_simple = meteo_simple.dropna(axis=0, subset=['temp_max'])
meteo_simple = meteo_simple.dropna(axis=0, subset=['temp_min'])

### Fill up
meteo_simple['temp_avg_where'] = np.where(meteo_simple.temp_avg.isnull(),(meteo_simple.temp_max+meteo_simple.temp_min)/2,meteo_simple.temp_avg)

### Create a boolean wind column
meteo_simple['wind_all'] = np.where(meteo_simple.wind_avgspeed.isnull(), '0', '1')

### Remove remaining NaN
meteo_simple['wind_avgspeed'] = meteo_simple['wind_avgspeed'].fillna(0)
meteo_simple['wind_fastest_2min'] = meteo_simple['wind_fastest_2min'].fillna(0)
meteo_simple['rain_prcp'] = meteo_simple['rain_prcp'].fillna(0)

### Drop 'temp_avg' and replace it by 'temp_avg_where'

meteo_simple.drop(['temp_avg'], axis=1,inplace=True)
meteo_simple.rename(columns={'temp_avg_where': 'temp_avg'},inplace=True)
meteo_simple.isnull().sum() 

date                 0
dust_sand            0
rain_prcp            0
smoke_haze           0
station_code         0
station_lat          0
station_lon          0
station_name         0
temp_max             0
temp_min             0
thunder              0
wind_avgspeed        0
wind_fastest_2min    0
fog_all              0
snow_all             0
rain_all             0
wind_dmg             0
temp_avg             0
wind_all             0
dtype: int64

###3.3. Riopa Data Cleaning: ###
There are some NaN and NaT values in the riopa dataset. NaT are simply due to the fact that no measurement of the particular pollutant was made in the respective row. That is fine because the main 'date' column has no missing values. The 9 missing temp*_*mean can be recalculated using temp*_*min and temp*_*max. Rows with missing pm25 will have to be dropped.
As part of the sampling is done on people, which is of no interest in this project, the riopa dataset has to be subset on airtype (= indoor and outdoor).

In [334]:
riopa.isnull().sum()

linkid                       0
sampleid                     0
homeid                       0
airtype                      0
pm25                        34
validity                   724
comments_pm25               16
date_start_pm25              0
date_end_pm25                0
date                         0
landuse_class              265
tempid                     205
visitnumber                205
date_temp                  205
location                   205
ambient_temp_c             205
ambient_rh                 205
airexrate                  359
comment_aer                830
date_start_aer             359
date_end_aer               359
temp_dry                    47
dew_point                   47
temp_wet                    47
rh                          47
census_group_block_code    265
home_lat                   350
home_long                  350
geoid                      350
dtype: int64

In [335]:
### Cleaning riopa

### Drop rows with missing pm25
riopa.dropna(subset=['pm25'],inplace=True)

### Replacing nan by 0
riopa['comment_aer'] = riopa['comment_aer'].fillna(0)
riopa['airexrate'] = riopa['airexrate'].fillna(0)
riopa['ambient_rh'] = riopa['ambient_rh'].fillna(0)
riopa['ambient_temp_c'] = riopa['ambient_temp_c'].fillna(0)
riopa['location'] = riopa['location'].fillna(0)
riopa['visitnumber'] = riopa['visitnumber'].fillna(0)
riopa['tempid'] = riopa['tempid'].fillna(0)
riopa['validity'] = riopa['validity'].fillna(0)
riopa['comments_pm25'] = riopa['comments_pm25'].fillna(0)
riopa['landuse_class'] = riopa['landuse_class'].fillna('tbd')

### subset to keep only interesting columns (i.e. removing multiple dates)
riopa_slim=riopa.drop(['validity','comments_pm25','tempid','visitnumber','date_temp','location','comment_aer','date_start_aer','date_end_aer'], axis=1,inplace=False)

### It looks good
riopa_slim.isnull().sum()

linkid                       0
sampleid                     0
homeid                       0
airtype                      0
pm25                         0
date_start_pm25              0
date_end_pm25                0
date                         0
landuse_class                0
ambient_temp_c               0
ambient_rh                   0
airexrate                    0
temp_dry                    46
dew_point                   46
temp_wet                    46
rh                          46
census_group_block_code    254
home_lat                   332
home_long                  332
geoid                      332
dtype: int64

In [336]:
### subset outdoor, indoor and leaving 'person' behind
riopa_outdoor=riopa_slim.loc[riopa.airtype=='OUTDOOR']
riopa_indoor=riopa_slim.loc[riopa.airtype=='INDOOR']
riopa_outdoor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 8 to 833
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   linkid                   126 non-null    object        
 1   sampleid                 126 non-null    int64         
 2   homeid                   126 non-null    object        
 3   airtype                  126 non-null    object        
 4   pm25                     126 non-null    float64       
 5   date_start_pm25          126 non-null    datetime64[ns]
 6   date_end_pm25            126 non-null    datetime64[ns]
 7   date                     126 non-null    datetime64[ns]
 8   landuse_class            126 non-null    object        
 9   ambient_temp_c           126 non-null    float64       
 10  ambient_rh               126 non-null    float64       
 11  airexrate                126 non-null    float64       
 12  temp_dry                 125 non-nul

###3.4. Air Quality Data Cleaning: ###

There are no null values in the 'frepa*_*pollutant' individual dataframes. There are null values in the merged epa dataframe because each air quality station may not record all the pollutants. That is expected and fine.
It looks like there are no null values in the 'frtamis*_*pollutant' individual frames but this is because all the null values are question marks. The function 'theanswer' takes a dataframe and a column name to drop the rows for which the column contains a question mark and convert the column into a numerical column. 

In [337]:
### frtamis_pb displays no null values
frtamis_pb.isnull().sum()

city          0
site_code     0
site_name     0
site_lat      0
site_lon      0
start_hour    0
start_time    0
duration      0
pb_24hr       0
date          0
dtype: int64

In [338]:
### although frtamis_pb contains ? in the pb_24hr column
frtamis_pb.head()

,city,site_code,site_name,site_lat,site_lon,start_hour,start_time,duration,pb_24hr,date
0,Houston,482011035,Clinton,29.733726,-95.257593,0,00:00,24 HOURS,?,2008-01-01
1,Houston,482011035,Clinton,29.733726,-95.257593,0,00:00,24 HOURS,.003,2008-01-01
2,Houston,482010066,Houston Westhollow,29.723333,-95.635833,0,00:00,24 HOURS,?,2008-01-01
3,Houston,482010047,Lang,29.834167,-95.489167,0,00:00,24 HOURS,?,2008-01-01
4,Houston,482010024,Houston Aldine,29.901036,-95.326137,0,00:00,24 HOURS,?,2008-01-01


In [339]:
### remove all question marks with the function 'theanswer'

def theanswer(df,col):
    ''' function drops rows where col has a ?'''
    dfcond=df[df[col]=='?'].index
    df.drop(dfcond,inplace=True)
    ''' function converts column to numeric'''    
    df[col]=pd.to_numeric(df[col])
    return df

ctamis_pm10=theanswer(frtamis_pm10_24hr,'pm10_24hr')
ctamis_pm10=theanswer(frtamis_pm10_24hr,'pm10_total_24hr')
ctamis_pm10=theanswer(frtamis_pm10_24hr,'pm10_minus_pm2_5_24hr')
ctamis_pm25=theanswer(frtamis_pm25,'pm25_24hr')
ctamis_pb=theanswer(frtamis_pb,'pb_24hr')

### naming dataframes without ? in the data as "clean" like the dataframes above
ctamis_ozone=mfrtamis_ozone
ctamis_no2=frtamis_no2
ctamis_so2=frtamis_so2
ctamis_co=frtamis_co

C:\Users\Anne\anaconda3\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Anne\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\Anne\anaconda3\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Anne\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; return

##4. Wrapping up:##

The datasets are ready for EDA.The method .describe() provides basics statistics for each dataset or individual columns. Additional work is required to get an understanding of the spatial distribution of the data. The geospatial work is done in another notebook and will be introduced in the EDA notebook.


Now it is time to save all the dataframes under '00*_*SavedDataFrames'.

In [348]:
### Print out all DSs for EDA

epa_stations.to_excel(path_header+path_df+'stations_epa.xlsx',sheet_name='epa_stations')
tamis_stations.to_excel(path_header+path_df+'stations_tamis.xlsx',sheet_name='tamis_stations')
allstations.to_excel(path_header+path_df+'stations_epatamis.xlsx',sheet_name='epa_tamis')

meteo_stations=meteo_simple[['station_name','station_lat','station_lon']].drop_duplicates()
meteo_stations.to_excel(path_header+path_df+'stations_all.xlsx',sheet_name='meteo')
meteo_stations.to_excel(path_header+path_df+'stations_meteo.xlsx',sheet_name='meteo')

riopa_slim.to_excel(path_header+path_df+'riopa_slim.xlsx',sheet_name='riopa_slim')
riopa_outdoor.to_excel(path_header+path_df+'riopa_outdoor.xlsx',sheet_name='riopa_outdoor')
riopa_indoor.to_excel(path_header+path_df+'riopa_indoor.xlsx',sheet_name='riopa_indoor')

meteo_simple.to_excel(path_header+path_df+'meteo_simple.xlsx',sheet_name='meteo_simple')


frepa_ozone.to_excel(path_header+path_df+'epa_ozone.xlsx',sheet_name='epa_ozone')
frepa_co.to_excel(path_header+path_df+'epa_co.xlsx',sheet_name='epa_co')
frepa_no2.to_excel(path_header+path_df+'epa_no2.xlsx',sheet_name='epa_no2')
frepa_so2.to_excel(path_header+path_df+'epa_so2.xlsx',sheet_name='epa_so2')
frepa_pm10.to_excel(path_header+path_df+'epa_pm10.xlsx',sheet_name='epa_pm10')
frepa_pm25.to_excel(path_header+path_df+'epa_pm25.xlsx',sheet_name='epa_pm25')
frepa_pb.to_excel(path_header+path_df+'epa_pb.xlsx',sheet_name='epa_pb')
epa.to_excel(path_header+path_df+'epa.xlsx',sheet_name='d_wrangling')

ctamis_ozone.to_excel(path_header+path_df+'tamis_ozone.xlsx',sheet_name='tamis_ozone')
ctamis_co.to_excel(path_header+path_df+'tamis_co.xlsx',sheet_name='tamis_co')
ctamis_no2.to_excel(path_header+path_df+'tamis_no2.xlsx',sheet_name='tamis_no2')
ctamis_so2.to_excel(path_header+path_df+'tamis_so2.xlsx',sheet_name='tamis_so2')
ctamis_pm10.to_excel(path_header+path_df+'tamis_pm10.xlsx',sheet_name='tamis_pm10')
ctamis_pm25.to_excel(path_header+path_df+'tamis_pm25.xlsx',sheet_name='tamis_pm25')
ctamis_pb.to_excel(path_header+path_df+'tamis_pb.xlsx',sheet_name='tamis_pb')
